In [ ]:
import sys
import os
import time


try:
    lib_path = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', 'lib'))
except NameError:
    lib_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'lib'))

sys.path.insert(0, lib_path)

from Apriori import AprioriAlgorithm
from FPGrowth import FPGrowthAlgorithm
from RuleGenerator import RulesGenerator

In [ ]:
def create_sample_data():
    """Створює приклад даних для тестування"""
    transactions = [
        ['хліб', 'молоко'],
        ['хліб', 'підгузники', 'пиво', 'яйця'],
        ['молоко', 'підгузники', 'пиво', 'кола'],
        ['хліб', 'молоко', 'підгузники', 'пиво'],
        ['хліб', 'молоко', 'підгузники', 'кола'],
        ['молоко', 'підгузники', 'пиво'],
        ['хліб', 'молоко'],
        ['хліб', 'підгузники', 'пиво'],
        ['хліб', 'молоко', 'кола'],
        ['молоко', 'підгузники', 'кола']
    ]
    return transactions

In [ ]:
def demo_apriori():
    """Демонстрація роботи алгоритму Apriori"""
    print("=" * 60)
    print("ДЕМОНСТРАЦІЯ АЛГОРИТМУ APRIORI")
    print("=" * 60)
    
    transactions = create_sample_data()
    print(f"Кількість транзакцій: {len(transactions)}")
    print("Приклад транзакцій:")
    for i, transaction in enumerate(transactions[:3]):
        print(f"  {i+1}: {transaction}")
    print("  ...")
    print()
    
    apriori = AprioriAlgorithm(min_support=0.3)
    frequent_itemsets = apriori.fit(transactions)
    
    print("ЧАСТІ НАБОРИ ЕЛЕМЕНТІВ:")
    for k, itemsets in frequent_itemsets.items():
        print(f"\n{k}-елементні набори:")
        for itemset, count in itemsets.items():
            support = count / len(transactions)
            items_str = ', '.join(sorted(itemset))
            print(f"  {{{items_str}}}: підтримка = {support:.3f} (кількість = {count})")
    
    print("\n" + "=" * 60)
    print("АСОЦІАТИВНІ ПРАВИЛА (APRIORI)")
    print("=" * 60)
    
    rules_gen = RulesGenerator(min_confidence=0.6)
    rules = rules_gen.generate_rules(apriori)
    
    if rules:
        rules_gen.print_rules(top_n=10)
    else:
        print("Не знайдено правил з заданими параметрами")
    
    return apriori, rules

In [ ]:
def demo_fp_growth():
    """Демонстрація роботи алгоритму FP-Growth"""
    print("\n" + "=" * 60)
    print("ДЕМОНСТРАЦІЯ АЛГОРИТМУ FP-GROWTH")
    print("=" * 60)
    
    transactions = create_sample_data()
    
    fp_growth = FPGrowthAlgorithm(min_support=0.3)
    frequent_itemsets = fp_growth.fit(transactions)
    
    print("ЧАСТІ НАБОРИ ЕЛЕМЕНТІВ:")
    for k, itemsets in frequent_itemsets.items():
        print(f"\n{k}-елементні набори:")
        for itemset, count in itemsets.items():
            support = count / len(transactions)
            items_str = ', '.join(sorted(itemset))
            print(f"  {{{items_str}}}: підтримка = {support:.3f} (кількість = {count})")
    
    print("\n" + "=" * 60)
    print("АСОЦІАТИВНІ ПРАВИЛА (FP-GROWTH)")
    print("=" * 60)
    
    rules_gen = RulesGenerator(min_confidence=0.6)
    rules = rules_gen.generate_rules(fp_growth)
    
    if rules:
        rules_gen.print_rules(top_n=10)
    else:
        print("Не знайдено правил з заданими параметрами")
    
    return fp_growth, rules

In [ ]:
def compare_algorithms():
    """Порівняння результатів двох алгоритмів"""
    print("\n" + "=" * 60)
    print("ПОРІВНЯННЯ АЛГОРИТМІВ")
    print("=" * 60)
    
    transactions = create_sample_data()
    
    # Apriori
    apriori = AprioriAlgorithm(min_support=0.3)
    apriori_itemsets = apriori.fit(transactions)
    
    # FP-Growth
    fp_growth = FPGrowthAlgorithm(min_support=0.3)
    fp_growth_itemsets = fp_growth.fit(transactions)
    
    print("Кількість знайдених частих наборів:")
    
    for k in range(1, max(max(apriori_itemsets.keys()), max(fp_growth_itemsets.keys())) + 1):
        apriori_count = len(apriori_itemsets.get(k, {}))
        fp_growth_count = len(fp_growth_itemsets.get(k, {}))
        print(f"  {k}-елементні: Apriori = {apriori_count}, FP-Growth = {fp_growth_count}")
    
    # Перевіряємо, чи результати однакові
    all_apriori = set()
    for itemsets in apriori_itemsets.values():
        all_apriori.update(itemsets.keys())
    
    all_fp_growth = set()
    for itemsets in fp_growth_itemsets.values():
        all_fp_growth.update(itemsets.keys())
    
    if all_apriori == all_fp_growth:
        print("\n✓ Результати обох алгоритмів однакові!")
    else:
        print("\n✗ Результати алгоритмів відрізняються")
        print(f"Тільки в Apriori: {all_apriori - all_fp_growth}")
        print(f"Тільки в FP-Growth: {all_fp_growth - all_apriori}")

In [ ]:
demo_apriori()

In [ ]:
demo_fp_growth()

In [ ]:
compare_algorithms()

In [ ]:
from mlxtend.frequent_patterns import apriori
import pandas as pd

transactions = []
with open(r'..\\res\groceries.csv', 'r', encoding='utf-8') as f:
    for line in f:
        transaction = line.strip().split(',')
        transactions.append(transaction)

min_support = 0.01

start = time.time()
apriori_custom = AprioriAlgorithm(min_support=min_support)
freq_itemsets_custom = apriori_custom.fit(transactions)

n_trans = len(transactions)
custom_supports = {
    itemset: count / n_trans
    for k, itemsets in freq_itemsets_custom.items()
    for itemset, count in itemsets.items()
}
end = time.time()
print(f"Час виконання мого AprioriAlgorithm: {end - start:.4f} секунд")

all_items = sorted({item for transaction in transactions for item in transaction})
encoded_vals = []
for transaction in transactions:
    encoded_vals.append({item: (item in transaction) for item in all_items})
df = pd.DataFrame(encoded_vals)

start = time.time()
freq_itemsets_mlxtend = apriori(df, min_support=min_support, use_colnames=True)
mlxtend_supports = {
    frozenset(row['itemsets']): row['support']
    for _, row in freq_itemsets_mlxtend.iterrows()
}
end = time.time()
print(f"Час виконання mlxtend apriori: {end - start:.4f} секунд")

print("Порівняння частих наборів і підтримки:\n")

all_itemsets = set(custom_supports.keys()) | set(mlxtend_supports.keys())

for itemset in sorted(all_itemsets, key=lambda x: (len(x), sorted(x))):
    support_custom = custom_supports.get(itemset, None)
    support_mlxtend = mlxtend_supports.get(itemset, None)
    
    if support_custom is None:
        print(f"{set(itemset)}: Відсутній у власному Apriori, mlxtend: {support_mlxtend:.2f}")
    elif support_mlxtend is None:
        print(f"{set(itemset)}: Є у моєму Apriori: {support_custom:.2f}, відсутній у mlxtend")
    else:
        diff = abs(support_custom - support_mlxtend)
        status = "OK" if diff < 1e-6 else f"Відмінність {diff:.4f}"
        print(f"{set(itemset)}: Твій: {support_custom:.2f}, mlxtend: {support_mlxtend:.2f} -> {status}")

In [ ]:
from mlxtend.frequent_patterns import fpgrowth
import pandas as pd

transactions = []
with open(r'..\\res\groceries.csv', 'r', encoding='utf-8') as f:
    for line in f:
        transaction = line.strip().split(',')
        transactions.append(transaction)

min_support = 0.01
n_trans = len(transactions)

start = time.time()
fp_custom = FPGrowthAlgorithm(min_support=min_support)
freq_itemsets_custom = fp_custom.fit(transactions)

custom_supports = {
    itemset: count / n_trans
    for k, itemsets in freq_itemsets_custom.items()
    for itemset, count in itemsets.items()
}
end = time.time()
print(f"Час виконання мого FP-Growth: {end - start:.4f} секунд")

all_items = sorted({item for transaction in transactions for item in transaction})
encoded_vals = []
for transaction in transactions:
    encoded_vals.append({item: item in transaction for item in all_items})
df = pd.DataFrame(encoded_vals)

start = time.time()
freq_itemsets_mlxtend = fpgrowth(df, min_support=min_support, use_colnames=True)
mlxtend_supports = {
    frozenset(row['itemsets']): row['support']
    for _, row in freq_itemsets_mlxtend.iterrows()
}
end = time.time()
print(f"Час виконання mlxtend fpgrowth: {end - start:.4f} секунд")

print("\nПорівняння частих наборів і підтримки:\n")

all_itemsets = set(custom_supports.keys()) | set(mlxtend_supports.keys())

for itemset in sorted(all_itemsets, key=lambda x: (len(x), sorted(x))):
    support_custom = custom_supports.get(itemset, None)
    support_mlxtend = mlxtend_supports.get(itemset, None)

    if support_custom is None:
        print(f"{set(itemset)}: Відсутній у моєї реалізації, mlxtend: {support_mlxtend:.2f}")
    elif support_mlxtend is None:
        print(f"{set(itemset)}: Є у моїй реалізації: {support_custom:.2f}, відсутній у mlxtend")
    else:
        diff = abs(support_custom - support_mlxtend)
        status = "OK" if diff < 1e-6 else f"Відмінність {diff:.4f}"
        print(f"{set(itemset)}: Твій: {support_custom:.2f}, mlxtend: {support_mlxtend:.2f} -> {status}")
